# Importing my own Books from Goodreads export tool
Goodreads export using: https://www.goodreads.com/review/import


In [1]:
# set working directory to same place ass app.py to import programs the same way as the app
import os
current_directory = os.getcwd()
if 'notebooks' in current_directory:
    parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
    os.chdir(parent_directory)
os.getcwd()

'/Users/elisealstad/Desktop/Code/mybook-dashboard'

In [ ]:
# Super fast!!! 

In [2]:
# Improt packages
import pandas as pd
import json 
import numpy as np

# Import functions from apps folder
from apps.collect_data import *

pd.set_option('max_colwidth', 50)
pd.set_option('display.max_columns', 80)

In [3]:
mybooksgr = pd.read_csv("assets/goodreads_library_export.csv")
mybooksgr = mybooksgr.rename(columns=lambda x: x.replace(' ', '_'))


In [20]:
import requests
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import nest_asyncio

async def get_book_info_async(session, book_name, author_name, api_key):
    base_url = 'https://www.googleapis.com/books/v1/volumes'
    params = {
        'q': f'intitle:{book_name}+inauthor:{author_name}',
        'key': api_key,
        'maxResults': 1,
        'fields': 'items(volumeInfo/title,volumeInfo/authors,volumeInfo/publishedDate,volumeInfo/description,volumeInfo/industryIdentifiers,volumeInfo/pageCount,volumeInfo/categories,volumeInfo/averageRating,volumeInfo/ratingsCount,volumeInfo/language)'
    }

    async with session.get(base_url, params=params) as response:
        if response.status == 200:
            data = await response.json()
            if 'items' in data and len(data['items']) > 0:
                return data['items'][0]['volumeInfo']
            print('request one done')
        else:
            print(f"Error {response.status}: {await response.text()}")
            return None

async def get_book_info(book_name, author_name, api_key):
    async with aiohttp.ClientSession() as session:
        return await get_book_info_async(session, book_name, author_name, api_key)

def book_info_add(df, api_key):
    async def get_book_info_wrapper(row):
        book_name = row['Title']
        author_name = row['Author']
        return await get_book_info(book_name, author_name, api_key)

    # Use nest_asyncio to allow running asyncio in a notebook
    nest_asyncio.apply()

    # Create an event loop
    loop = asyncio.get_event_loop()

    # Run the asynchronous code
    # tasks = [get_book_info_wrapper(row) for _, row in df.iterrows()]
    # delay = 0.2
    # book_infos = loop.run_until_complete(asyncio.gather(*tasks))
    
        # Introduce a delay of 0.1 seconds between requests
    delay = 0.1
    book_infos = asyncio.run(asyncio.gather(*[get_book_info_wrapper(row) for _, row in df.iterrows()]))


    combined_book_info = pd.DataFrame()

    for book_info in book_infos:
        if book_info:
            authors = book_info.get('authors', [np.nan])
            publish_date = book_info.get('publishedDate', np.nan)
            description = book_info.get('description', np.nan)
            identifiers = book_info.get('industryIdentifiers', [])
            isbn = identifiers[0]['identifier'] if identifiers else np.nan
            page_count = book_info.get('pageCount', np.nan)
            categories = book_info.get('categories', [np.nan])
            average_rating = book_info.get('averageRating', np.nan)
            rating_count = book_info.get('ratingsCount', np.nan)
            language = book_info.get('language', np.nan)

            book_info_df = pd.DataFrame({
                'Title': [book_info.get('title', np.nan)],
                'Author(s)': [", ".join(map(str, authors))],
                'Publish Date': [publish_date],
                'Description': [description],
                'ISBN': [isbn],
                'Page Count': [page_count],
                'Categories': [", ".join(map(str, categories))],
                'Average Rating': [average_rating],
                'Rating Count': [rating_count],
                'Language': [language]
            })
            combined_book_info = pd.concat([combined_book_info, book_info_df])

    # Reset the index of the combined DataFrame
    combined_book_info = combined_book_info.reset_index(drop=True)
    combined_book_info = combined_book_info.rename(columns=lambda x: x.replace(' ', '_'))

    return combined_book_info
result = book_info_add(mybooksgr.head(10), api_key)
print(result)



request one done
request one done
                                 Title        Author(s) Publish_Date  \
0                     Beautiful Malice    Rebecca James   2010-08-19   
1                     Writers & Lovers        Lily King   2020-03-03   
2                       The Guest List       Lucy Foley   2022-06-28   
3                    The Mystery Guest       Nita Prose   2023-11-28   
4                The Girl on the Train    Paula Hawkins   2016-07-12   
5                    Project Hail Mary        Andy Weir   2021-05-04   
6  The Subtle Art of Not Giving a F*ck      Mark Manson   2016-09-13   
7        The Lottery and Other Stories  Shirley Jackson   2009-10-01   

                                         Description           ISBN  \
0  So. Were you glad, deep down? Were you glad to...  9780571255306   
1  #ReadWithJenna Book Club Pick as Featured on T...  9780802148551   
2  A REESE'S BOOK CLUB PICK THE NEW YORK TIMES BE...     0063215381   
3  BOOK 2 OF 2: MOLLY THE MAID “P

# sleep funciton

In [17]:

import requests
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import nest_asyncio
import time

async def get_book_info_async(session, book_name, author_name, api_key):
    base_url = 'https://www.googleapis.com/books/v1/volumes'
    params = {
        'q': f'intitle:{book_name}+inauthor:{author_name}',
        'key': api_key,
        'maxResults': 1,
        'fields': 'items(volumeInfo/title,volumeInfo/authors,volumeInfo/publishedDate,volumeInfo/description,volumeInfo/industryIdentifiers,volumeInfo/pageCount,volumeInfo/categories,volumeInfo/averageRating,volumeInfo/ratingsCount,volumeInfo/language)'
    }

    async with session.get(base_url, params=params) as response:
        if response.status == 200:
            data = await response.json()
            if 'items' in data and len(data['items']) > 0:
                return data['items'][0]['volumeInfo']
            print('request one done')
        else:
            print(f"Error {response.status}: {await response.text()}")
            return None

async def get_book_info(book_name, author_name, api_key):
    async with aiohttp.ClientSession() as session:
        return await get_book_info_async(session, book_name, author_name, api_key)

async def book_info_add(df, api_key):
    async def get_book_info_wrapper(row):
        book_name = row['Title']
        author_name = row['Author']
        return await get_book_info(book_name, author_name, api_key)

    # Use nest_asyncio to allow running asyncio in a notebook
    nest_asyncio.apply()

    # Run the asynchronous code
    tasks = [get_book_info_wrapper(row) for _, row in df.iterrows()]

    # Run the asynchronous code
    results = await asyncio.gather(*tasks)  # Corrected placement of await
    print('finishe, creating dfd')
    combined_book_info = pd.DataFrame()

    # Extract the actual results from the list
    for book_info in results:
        if book_info:
            authors = book_info.get('authors', [np.nan])
            publish_date = book_info.get('publishedDate', np.nan)
            description = book_info.get('description', np.nan)
            identifiers = book_info.get('industryIdentifiers', [])
            isbn = identifiers[0]['identifier'] if identifiers else np.nan
            page_count = book_info.get('pageCount', np.nan)
            categories = book_info.get('categories', [np.nan])
            average_rating = book_info.get('averageRating', np.nan)
            rating_count = book_info.get('ratingsCount', np.nan)
            language = book_info.get('language', np.nan)

            book_info_df = pd.DataFrame({
                'Title': [book_info.get('title', np.nan)],
                'Author(s)': [", ".join(map(str, authors))],
                'Publish Date': [publish_date],
                'Description': [description],
                'ISBN': [isbn],
                'Page Count': [page_count],
                'Categories': [", ".join(map(str, categories))],
                'Average Rating': [average_rating],
                'Rating Count': [rating_count],
                'Language': [language]
            })
            combined_book_info = pd.concat([combined_book_info, book_info_df])

    # Reset the index of the combined DataFrame
    combined_book_info = combined_book_info.reset_index(drop=True)
    combined_book_info = combined_book_info.rename(columns=lambda x: x.replace(' ', '_'))
   
from apps.api import api_key
result = await book_info_add(mybooksgr.sample(5), api_key)  # Use await when calling the async function
print(result)

In [ ]:

import requests
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import nest_asyncio
import time

async def get_book_info_async(session, book_name, author_name, api_key):
    base_url = 'https://www.googleapis.com/books/v1/volumes'
    params = {
        'q': f'intitle:{book_name}+inauthor:{author_name}',
        'key': api_key,
        'maxResults': 1,
        'fields': 'items(volumeInfo/title,volumeInfo/authors,volumeInfo/publishedDate,volumeInfo/description,volumeInfo/industryIdentifiers,volumeInfo/pageCount,volumeInfo/categories,volumeInfo/averageRating,volumeInfo/ratingsCount,volumeInfo/language)'
    }

    async with session.get(base_url, params=params) as response:
        if response.status == 200:
            data = await response.json()
            if 'items' in data and len(data['items']) > 0:
                return data['items'][0]['volumeInfo']
            print('request one done')
        else:
            print(f"Error {response.status}: {await response.text()}")
            return None

async def get_book_info(book_name, author_name, api_key):
    async with aiohttp.ClientSession() as session:
        return await get_book_info_async(session, book_name, author_name, api_key)

async def book_info_add(df, api_key):
    async def get_book_info_wrapper(row):
        book_name = row['Title']
        author_name = row['Author']
        return await get_book_info(book_name, author_name, api_key)

    # Use nest_asyncio to allow running asyncio in a notebook
    nest_asyncio.apply()

    # Run the asynchronous code
    tasks = [get_book_info_wrapper(row) for _, row in df.iterrows()]

    # Run the asynchronous code
    results = await asyncio.gather(*tasks)  # Corrected placement of await
    print('finishe, creating dfd')
    combined_book_info = pd.DataFrame()

    # Extract the actual results from the list
    for book_info in results:
        if book_info:
            authors = book_info.get('authors', [np.nan])
            publish_date = book_info.get('publishedDate', np.nan)
            description = book_info.get('description', np.nan)
            identifiers = book_info.get('industryIdentifiers', [])
            isbn = identifiers[0]['identifier'] if identifiers else np.nan
            page_count = book_info.get('pageCount', np.nan)
            categories = book_info.get('categories', [np.nan])
            average_rating = book_info.get('averageRating', np.nan)
            rating_count = book_info.get('ratingsCount', np.nan)
            language = book_info.get('language', np.nan)

            book_info_df = pd.DataFrame({
                'Title': [book_info.get('title', np.nan)],
                'Author(s)': [", ".join(map(str, authors))],
                'Publish Date': [publish_date],
                'Description': [description],
                'ISBN': [isbn],
                'Page Count': [page_count],
                'Categories': [", ".join(map(str, categories))],
                'Average Rating': [average_rating],
                'Rating Count': [rating_count],
                'Language': [language]
            })
            combined_book_info = pd.concat([combined_book_info, book_info_df])

    # Reset the index of the combined DataFrame
    combined_book_info = combined_book_info.reset_index(drop=True)
    combined_book_info = combined_book_info.rename(columns=lambda x: x.replace(' ', '_'))

    return combined_book_info


if __name__ == "__main__":
    asyncio.run(book_info_add())

In [15]:
from apps.api import api_key
import asyncio
from apps.async_googleapi import book_info_add
result = await book_info_add(mybooksgr.sample(5), api_key) # Use await when calling the async function
print(result)

                            Title        Author(s) Publish_Date  \
0              The Couple at No 9   Claire Douglas   2021-09-16   
1                      After Dark  Haruki Murakami   2011-10-10   
2  The Astonishing Color of After   Emily X.R. Pan   2019-03-19   
3             Visual Intelligence    Amy E. Herman   2016-05-03   
4                    Cursed Bunny       Bora Chung   2022-12-06   

                                         Description           ISBN  \
0  It was the house of their dreams. Now it's the...  9781405943413   
1  Reality bends all the more acutely with lack o...  9781448103638   
2  "Emily X.R. Pan's brilliantly crafted, harrowi...     0316464015   
3  An engrossing guide to seeing—and communicatin...  9780544381063   
4  FINALIST FOR THE 2023 NATIONAL BOOK AWARD IN T...  9781643755007   

   Page_Count           Categories  Average_Rating  Rating_Count Language  
0         338              Fiction             NaN           NaN       en  
1         210     

In [33]:
# Merge dataframes 
mybooks = pd.merge(mybooksgr,
                     apimydf,
                     on='Title', 
                     suffixes = ('_Goodreads', '_GoogleBooks'), 
                     how='left')

# Data cleaning books df
- page count categories
- filter if book is read of not 

In [35]:
# Page count category variable

def categorize_pages(number_of_pages):
    if number_of_pages >= 100 and number_of_pages <= 249:
        return '100-249'
    elif number_of_pages >= 250 and number_of_pages <= 349:
        return '250-349'
    elif number_of_pages >= 350 and number_of_pages <= 449:
        return '350-449'
    elif number_of_pages >= 450 and number_of_pages <= 599:
        return '450-599'
    elif number_of_pages >= 600 and number_of_pages <= 749:
        return '600-749'
    elif number_of_pages >= 750 and number_of_pages <= 999:
        return '750-999'
    else:
        return '1000+'

# Apply the categorize_pages function to create the 'Page_Cat' column
mybooks['Page_Cat'] = mybooks['Number_of_Pages'].apply(categorize_pages)

# Define the desired order of categories
category_order = ['100-249', '250-349', '350-449', '450-599', '600-749', '750-999', '1000+']

# Convert the 'Page_Cat' column to a categorical variable with the specified order
mybooks['Page_Cat'] = pd.Categorical(mybooks['Page_Cat'], categories=category_order, ordered=True)


In [36]:
# drop duplicates
mybooks = mybooks.drop_duplicates(subset=['Title', 'Author'])

In [38]:
# Create year and quarter read variable 
#  Impute data_added where date_read  is na
mybooks['Date_Read'] = np.where(mybooks['Date_Read'].isnull() & mybooks['Read_Count']==1, mybooks['Date_Added'], mybooks['Date_Read'])

# Convert 'Date_Read' column to datetime type
mybooks['Date_Read'] = pd.to_datetime(mybooks['Date_Read'], format='mixed')

# Extract year and quarter from 'Date_Read' column
mybooks['Year'] = mybooks['Date_Read'].dt.year
mybooks['Quarter'] = mybooks['Date_Read'].dt.quarter

# Create a new column combining year and quarter
mybooks['Year_Quarter'] = np.where(mybooks['Date_Read'].notnull(), mybooks['Year'].astype(str) + '-Q' + mybooks['Quarter'].astype(str), np.nan)
# Replace '.0' in the Year_Quarter column with an empty string
mybooks['Year_Quarter'] = mybooks['Year_Quarter'].fillna('').str.replace('.0', '')

# Convert Year_Quarter to categorical variable
mybooks['Year_Quarter'] = pd.Categorical(mybooks['Year_Quarter'], ordered=True)

In [41]:
# filter na in publication year and make column publication year integer 
mybooks['Original_Publication_Year'] = mybooks['Original_Publication_Year'].fillna( 0)
mybooks['Original_Publication_Year'] = mybooks['Original_Publication_Year'].astype(int)

In [43]:
# Making sure all na is set as np.nan and not as a string variable (had this issue with one variable)
import numpy as np
mybooks = mybooks.replace('nan', np.nan)
mybooks = mybooks.replace('NaN', np.nan)

In [44]:
import pickle
mybooks.to_pickle("assets/my_books.pkl")

# Collect book topics 

In [21]:
# Collect topics for my own books from OLapi
my_topics = get_book_topics(mybooks)

In [22]:
# Writes the topics as 
with open("assets/my_topics.json", "w") as outfile:
    json.dump(my_topics, outfile)